In [1]:
from recipe_scrapers import scrap_me
from bs4 import BeautifulSoup
import urllib
import threading
import time
import csv
import pandas
from multiprocessing.dummy import Pool as ThreadPool 

In [1]:
cuisine = {
'brazilian':'https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=', 
'cajun-and-creole':'https://www.allrecipes.com/recipes/272/us-recipes/cajun-and-creole/?page=',
'southern':'https://www.allrecipes.com/recipes/15876/us-recipes/southern/?page=',
'moroccan':'https://www.allrecipes.com/recipes/1827/world-cuisine/african/north-african/moroccan/?page=',
'filipino':'https://www.allrecipes.com/recipes/696/world-cuisine/asian/filipino/?page=',
'irish':'https://www.allrecipes.com/recipes/706/world-cuisine/european/uk-and-ireland/irish/?page=',
'british':'https://www.allrecipes.com/recipes/705/world-cuisine/european/uk-and-ireland/english/?page=',
'russian':'https://www.allrecipes.com/recipes/716/world-cuisine/european/eastern-european/russian/?page=',
'french':'https://www.allrecipes.com/recipes/721/world-cuisine/european/french/?page=',
'greek':'https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/?page=',
'jamaican':'https://www.allrecipes.com/recipes/710/world-cuisine/latin-american/caribbean/jamaican/?page=',
'korean':'https://www.allrecipes.com/recipes/700/world-cuisine/asian/korean/?page=',
'spanish':'https://www.allrecipes.com/recipes/726/world-cuisine/european/spanish/?page=',
'thai':'https://www.allrecipes.com/recipes/702/world-cuisine/asian/thai/?page=',
'vietnamese':'https://www.allrecipes.com/recipes/703/world-cuisine/asian/vietnamese/?page=',
'indian':'https://www.allrecipes.com/recipes/233/world-cuisine/asian/indian/?page=',
'italian':'https://www.allrecipes.com/recipes/723/world-cuisine/european/italian/?page=',
'mexican':'https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/?=page',
'japanese':'https://www.allrecipes.com/recipes/699/world-cuisine/asian/japanese/?page='
}
# url = 'https://www.allrecipes.com/recipes/703/world-cuisine/asian/vietnamese/?page='

This cell works fine. It speeds up the scraping process by multithreading; however, if you run this it is highly likely that you'll be blocked by the target server.

In [3]:
# urls = []
# for i in range(1,700):
#     urls.append( url+str(i) )
    
# # Make the Pool of workers
# pool = ThreadPool(4) 
# # Open the urls in their own threads
# # and return the results
# results = pool.map(read_url, urls)
# #close the pool and wait for the work to finish 
# pool.close() 
# pool.join() 

Instead we use a slower process; however, a more effective one.

In [5]:
def augment_url(url):
    urls = []
    for i in range(1,2000):
        urls.append( url+str(i) )
    return urls

# augmentedUrls = augment_url(url)

In [6]:
def read_url(url):
    try:
        request = urllib.request
        html_doc = request.urlopen( url ).read()
        print("{}".format(url)+" scrapped!")
        return html_doc
    except:
        print("page does not exist {} end".format(url))
        return "end"
    
# recipePages = []
# for augmentedUrl in augmentedUrls:
#     res = read_url(augmentedUrl)
#     if (res=="end"): break
#     recipePages.append( res )

In [7]:
def readRecipeCards(recipePages):
    links = []
    for page in recipePages:
        soup = BeautifulSoup(page, 'html.parser')
        a_tags = soup.findAll("a", {"class": "fixed-recipe-card__title-link"})
        for a_tag in a_tags:
            links.append(a_tag['href'])
    return links


# recipeUrls = readRecipeCards( recipePages ) 

In [8]:
def extractText(urls,cuisine):
    recipe = []
    res = []

    for url in urls:
        res.append( read_url(url) )
        
    for data in res:
        soup = BeautifulSoup(data, 'html.parser')

        ingredients = soup.findAll("span", {"class": "recipe-ingred_txt added"})
        title = soup.findAll("h1",{"class":"recipe-summary__h1"})

        name = title[0].get_text()
        inge = []
        for ingredient in ingredients:
            inge.append( ingredient.get_text() )

        recipe.append({
            "name":name,
            "cuisine" : cuisine,
            "ingredient":inge
        })
    return recipe
    
# recipes = extractText(recipeUrls,cuisine)

In [9]:
def housekeelping(recipes,cuisine):
    res = pandas.DataFrame(recipes)
    res.to_pickle("allrecipe_"+cuisine)
    res.to_csv('allrecipe_'+cuisine+'.csv')
    
# housekeelping(recipes,"vietnamese")

In [10]:
for key in cuisine:
    augmentedUrls = augment_url(cuisine[key])
    
    recipePages = []
    for augmentedUrl in augmentedUrls:
        res = read_url(augmentedUrl)
        if (res=="end"): break
        recipePages.append( res )
        
    recipeUrls = readRecipeCards( recipePages )
    recipes = extractText(recipeUrls,key)
    housekeelping(recipes,key)


https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=1 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=2 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=3 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=4 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=5 scrapped!
https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=6 scrapped!
page does not exist https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/?page=7 end
https://www.allrecipes.com/recipe/149479/brazilian-style-moist-coconut-cake/ scrapped!
https://www.allrecipes.com/recipe/58647/brazilian-chicken-with-coconut-milk/ scrapped!
https://www.allrecipes.com/r

https://www.allrecipes.com/recipe/259263/red-grape-and-acai-caipirinha/ scrapped!
https://www.allrecipes.com/recipe/260580/easy-brazilian-pumpkin-coconut-dessert-with-few-ingredients/ scrapped!
https://www.allrecipes.com/recipe/256241/brazilian-christmas-rice/ scrapped!
https://www.allrecipes.com/recipe/254750/brazilian-crab-cakes-casquinha-de-siri/ scrapped!
https://www.allrecipes.com/recipe/254751/brazilian-grilled-salmon-with-mango-salsa/ scrapped!
https://www.allrecipes.com/recipe/255025/espetinho-de-queijo-coalho-com-abacaxi-brazilian-grilled-cheese-with-pineapple/ scrapped!
https://www.allrecipes.com/recipe/254724/caipirinha-de-limo-tradicional-classic-brazilian-cocktail/ scrapped!
https://www.allrecipes.com/recipe/254725/caipirinha-de-morango-com-kiwi-strawberry-kiwi-cocktail/ scrapped!
https://www.allrecipes.com/recipe/221436/brazilian-arroz-de-festa-confetti-rice/ scrapped!
https://www.allrecipes.com/recipe/221438/carrot-farofa/ scrapped!
https://www.allrecipes.com/recipe/2461

https://www.allrecipes.com/recipe/17016/mushroom-and-swiss-chicken/ scrapped!
https://www.allrecipes.com/recipe/25861/anns-shrimp-etouffee/ scrapped!
https://www.allrecipes.com/recipe/151366/costas-french-market-doughnuts-beignets/ scrapped!
https://www.allrecipes.com/recipe/38028/blackened-fish/ scrapped!
https://www.allrecipes.com/recipe/25667/cajun-crab-soup/ scrapped!
https://www.allrecipes.com/recipe/50026/cajun-chicken-lasagna/ scrapped!
https://www.allrecipes.com/recipe/199581/cajun-spiced-pork-chops/ scrapped!
https://www.allrecipes.com/recipe/18377/blackened-tuna/ scrapped!
https://www.allrecipes.com/recipe/48548/cajun-shrimp-orecchiette/ scrapped!
https://www.allrecipes.com/recipe/87079/quick-lemon-dijon-chicken/ scrapped!
https://www.allrecipes.com/recipe/12855/cajun-style-blackened-snapper/ scrapped!
page does not exist https://www.allrecipes.com/recipe/216909/stuffed-jalapeno-firecrackers/ end
https://www.allrecipes.com/recipe/87507/corn-and-crab-bisque/ scrapped!
https://

https://www.allrecipes.com/recipe/22732/crawfish-etouffee-like-maw-maw-used-to-make/ scrapped!
https://www.allrecipes.com/recipe/222581/kale-and-quinoa-with-creole-seasoning/ scrapped!
https://www.allrecipes.com/recipe/149780/cajun-roast-beef/ scrapped!
https://www.allrecipes.com/recipe/29459/shrimp-jambalaya/ scrapped!
https://www.allrecipes.com/recipe/12861/carolina-fish-cakes/ scrapped!
https://www.allrecipes.com/recipe/219644/kes-cajun-dirty-rice/ scrapped!
https://www.allrecipes.com/recipe/233398/summer-squash-and-sausage-stew/ scrapped!
https://www.allrecipes.com/recipe/220863/pressure-cooker-red-beans-and-sausage/ scrapped!
https://www.allrecipes.com/recipe/19936/jalapeno-hamburgers/ scrapped!
https://www.allrecipes.com/recipe/148765/real-new-orleans-style-bbq-shrimp/ scrapped!
https://www.allrecipes.com/recipe/147151/mamas-tomato-gravy/ scrapped!
https://www.allrecipes.com/recipe/213573/crawfish-cornbread/ scrapped!
https://www.allrecipes.com/recipe/44781/creole-mexican-catfish

https://www.allrecipes.com/recipe/13222/jambalaya-i/ scrapped!
https://www.allrecipes.com/recipe/29100/creole-macaroni-and-cheese/ scrapped!
https://www.allrecipes.com/recipe/151105/creole-pan-fried-flat-iron-steak/ scrapped!
https://www.allrecipes.com/recipe/172138/louisiana-crawfish-ya-ya-pasta/ scrapped!
https://www.allrecipes.com/recipe/55359/cajun-chicken-and-rice/ scrapped!
https://www.allrecipes.com/recipe/75923/rattlesnake-pasta/ scrapped!
https://www.allrecipes.com/recipe/86605/cajun-corn-soup/ scrapped!
https://www.allrecipes.com/recipe/216487/new-orleans-eggplant-casserole/ scrapped!
https://www.allrecipes.com/recipe/216886/jambalaya-pot-pie/ scrapped!
https://www.allrecipes.com/recipe/231833/spicy-cajun-cabbage/ scrapped!
https://www.allrecipes.com/recipe/239100/sausage-and-shrimp-gumbo/ scrapped!
https://www.allrecipes.com/recipe/53929/rice-cooker-crawfish-tails/ scrapped!
https://www.allrecipes.com/recipe/20678/ham-and-corn-beignets/ scrapped!
https://www.allrecipes.com/r

https://www.allrecipes.com/recipe/212892/alligator-animal-italian-bread/ scrapped!
https://www.allrecipes.com/recipe/213026/spicy-creamy-cajun-ham-and-black-eyed-peas-salad/ scrapped!
https://www.allrecipes.com/recipe/213555/cajun-style-eggs-benedict/ scrapped!
https://www.allrecipes.com/recipe/103307/shrimp-creole-pronto/ scrapped!
https://www.allrecipes.com/recipe/109881/cajun-crab-rangoon/ scrapped!
https://www.allrecipes.com/recipe/133040/teddys-duck-gumbo/ scrapped!
https://www.allrecipes.com/recipe/212767/shrimp-cheese-ball/ scrapped!
https://www.allrecipes.com/recipe/206680/crawfish-cakes/ scrapped!
https://www.allrecipes.com/recipe/197908/crawfish-boulettes/ scrapped!
https://www.allrecipes.com/recipe/169212/crawfish-macquechou/ scrapped!
https://www.allrecipes.com/recipe/157957/cajun-turkey-stuffing/ scrapped!
https://www.allrecipes.com/recipe/13875/sausage-and-oyster-stuffing/ scrapped!
https://www.allrecipes.com/recipe/22427/crawfish-etouffee-iii/ scrapped!
https://www.allre

https://www.allrecipes.com/recipe/246480/cracklins-deep-fried-pork-belly/ scrapped!


IndexError: list index out of range

In the end all dataframes are concatenated and saved as a file.

In [25]:
frameNames = []
for item in cuisine:
    frameNames.append("allrecipe_"+item)

frames = []
for name in frameNames:
    frames.append(pandas.read_pickle(name))
    
Allrecipie = pandas.concat(frames)
Allrecipie.to_csv('Allrecipe.csv')
Allrecipie.to_pickle("Allrecipe.pkl")

In [2]:
ingredients.head()

NameError: name 'ingredients' is not defined

# Preprocessing Data

In [28]:
import gensim 
import logging
import pandas
import pprint
from collections import Counter
recipies = pandas.read_pickle("Dataset/Allrecipe.pkl")

In [29]:
recipies.head()

,cuisine,ingredients,name
0,brazilian,"[ all purpose flour, baking powder, egg wh...",Brazilian-Style Moist Coconut Cake
1,brazilian,"[ ground cumin, ground cayenne pepper, gr...",Brazilian Chicken with Coconut Milk
2,brazilian,"[ cooked white rice, all purpose flour, eg...",Fried Rice Cakes (Brazilian-Style)
3,brazilian,"[ brown sugar, water, bananas peeled len...",Brazilian Whole Banana Pie
4,brazilian,"[ pound dry black beans, quarts water plus me...",Chef John's Brazilian Feijoada


In [30]:
import re as regex
def clean(ingredient):
    ingredient = [[regex.sub(r'\d|/|\([^)]*\)', '', x) for x in y] for y in ingredient]
    ingredient = [[regex.sub(r'tablespoons|teaspoons|teaspoon|fresh|tablespoon|chopped|cups|and|or|chopped|sliced|diced|cup|to|taste|:|,', '', x) for x in y] for y in ingredient]
    ingredient = [[x.replace('-',' ') for x in y] for y in ingredient]
    return ingredient

# recipies['ingredient'] = clean(recipies['ingredient'])
# recipies['cuisine'] = recipies['cuisine'].str.lower()
# recipies = recipies.rename(columns={'ingredient':'ingredients'})
recipies['cuisine'] = recipies['cuisine'].replace('cajun-and-creole','cajun_creole')
recipies['cuisine'] = recipies['cuisine'].replace('english','british')

In [30]:
wordCounter = Counter()
def func(row):
    for ingredient in row['ingredient']:
        wordCounter.update(ingredient.split())
    
recipies.apply(func,axis=1)
pprint.pprint(wordCounter)

Counter({'pepper': 7784,
         'ground': 6735,
         'salt': 6357,
         'oil': 5219,
         'garlic': 4901,
         'cheese': 4460,
         'onion': 4218,
         'black': 3889,
         'white': 3675,
         'can': 3625,
         'minced': 3475,
         'in': 3178,
         'sugar': 3062,
         'pound': 2918,
         'olive': 2891,
         'water': 2878,
         'dried': 2844,
         'green': 2794,
         'sauce': 2671,
         'cloves': 2664,
         'powder': 2614,
         'red': 2571,
         'cut': 2537,
         'chicken': 2530,
         'flour': 2464,
         'butter': 2463,
         'maes': 2264,
         'package': 2240,
         'shredded': 2128,
         'peeled': 2080,
         'large': 2048,
         'cream': 1839,
         'drained': 1747,
         'all': 1695,
         'purpose': 1694,
         'vegetable': 1657,
         'grated': 1644,
         'ounces': 1642,
         'milk': 1582,
         'pounds': 1561,
         'bell': 1559,
      

         'navy': 11,
         'Crust': 11,
         'Icing': 11,
         'Butter': 11,
         'rump': 11,
         'substitute': 11,
         'jelly': 11,
         'heated': 11,
         'mangos': 11,
         'bulbs': 11,
         'ladyfingers': 11,
         'Meat': 11,
         'Beef': 11,
         'poppy': 11,
         'hazelnut': 11,
         'gold': 11,
         'loosely': 11,
         'Olive': 11,
         'breads': 11,
         'zesty': 11,
         'jerk': 11,
         'sushi': 11,
         'galangal': 11,
         'lemongrass': 11,
         'Curry': 11,
         'meatballs': 11,
         'PHILADELPHIA': 11,
         'nutritional': 11,
         'crema': 11,
         'Tacos': 11,
         'bass': 10,
         'Chocolate': 10,
         'quality': 10,
         'rubbed': 10,
         'breadcrumbs': 10,
         'julienne': 10,
         'giblets': 10,
         'Swanson®': 10,
         'buttery': 10,
         'Kosher': 10,
         'canning': 10,
         'sriracha': 10,
         

         'Mizithra': 3,
         'bulgur': 3,
         'Simple': 3,
         'rennet': 3,
         'radiare': 3,
         'applewood': 3,
         'carp': 3,
         'drumettes': 3,
         'caramel': 3,
         'racks': 3,
         'kochujang': 3,
         'soybean': 3,
         'walnut': 3,
         'Slaw': 3,
         'kelp': 3,
         'wakame': 3,
         'tteok': 3,
         'bias': 3,
         'gyoza': 3,
         'cakes': 3,
         'Light': 3,
         'accding': 3,
         'directions': 3,
         'manchego': 3,
         'bird': 3,
         'Panang': 3,
         'roots': 3,
         'natural': 3,
         'Vietnamese': 3,
         'thoughly': 3,
         'Sriracha': 3,
         'creamer': 3,
         'Fresno': 3,
         'Gravy': 3,
         'pigeon': 3,
         'game': 3,
         'hens': 3,
         'Milk': 3,
         'chana': 3,
         'spreadable': 3,
         'rhubarb': 3,
         'Cabbage': 3,
         'pies': 3,
         'amaranth': 3,
         'moong': 3

         'kalamansi': 1,
         'calamansis': 1,
         'sinamak': 1,
         'Caramel': 1,
         'Sprinkle': 1,
         'Fried': 1,
         'leafy': 1,
         'ubod': 1,
         'bihon': 1,
         'cann': 1,
         'pala': 1,
         'misua': 1,
         'Colcannon': 1,
         'briskets': 1,
         'Guinness®': 1,
         'oatmeal': 1,
         'dye': 1,
         'crown': 1,
         'streaky': 1,
         "lamb's": 1,
         'Basting': 1,
         'Pickling': 1,
         'larger': 1,
         'Cned': 1,
         'Earl': 1,
         'unasted': 1,
         'sausagemeat': 1,
         'Pudding': 1,
         'cookie': 1,
         'cutters': 1,
         'crumpet': 1,
         "Pimm's": 1,
         'No.': 1,
         'foie': 1,
         'gras': 1,
         'Calimyrna': 1,
         'demerara': 1,
         'caramels': 1,
         'Cox': 1,
         'greasing': 1,
         'Scones': 1,
         'undyed': 1,
         'shtdough': 1,
         'Mushrooms': 1,
         'fil

         'ReaLime': 1,
         'Clama®': 1,
         'Cocktail': 1,
         'Ice': 1,
         'Rimming': 1,
         "Rose's®": 1,
         'Aioli': 1,
         'bloody': 1,
         'mary': 1,
         'Conmemativo': 1,
         'cascabel': 1,
         'mita': 1,
         "Europe's": 1,
         'Best®': 1,
         "Chef's": 1,
         'Spinach': 1,
         'Casbah®': 1,
         'Freekeh': 1,
         'iginal': 1,
         'Casserole': 1,
         'Charred': 1,
         'al': 1,
         'Pasr': 1,
         'bodies': 1,
         'Achiote': 1,
         'Vermont': 1,
         'Maple': 1,
         'Breakfast': 1,
         'guava': 1,
         'Sopes': 1,
         'cousa': 1,
         'yuca': 1,
         'patty': 1,
         'truffles': 1,
         'rBGH': 1,
         'Breast': 1,
         'rBST': 1,
         'Cranberry': 1,
         'Ins': 1,
         'Quinoa': 1,
         'pepitas': 1,
         'Dark': 1,
         'Cinnamon': 1,
         'Cardamom': 1,
         'hulls': 1,
      

In [26]:
recipies

,cuisine,ingredients,name
0,brazilian,"[ all purpose flour, baking powder, egg wh...",Brazilian-Style Moist Coconut Cake
1,brazilian,"[ ground cumin, ground cayenne pepper, gr...",Brazilian Chicken with Coconut Milk
2,brazilian,"[ cooked white rice, all purpose flour, eg...",Fried Rice Cakes (Brazilian-Style)
3,brazilian,"[ brown sugar, water, bananas peeled len...",Brazilian Whole Banana Pie
4,brazilian,"[ pound dry black beans, quarts water plus me...",Chef John's Brazilian Feijoada
5,brazilian,"[ olive oil, onion , cloves garlic minced, ...",Brazilian Rice
6,brazilian,"[ lime juice, ground cumin, paprika, mi...",Brazilian Fish Stew
7,brazilian,"[ butter, white sugar, can sweetened cond...",Brazilian Bom-Bocado (Egg Custard)
8,brazilian,"[ limes, water, white sugar, sweetened c...",Easy Brazilian Lemonade
9,brazilian,"[ pk spareribs, soy sauce, cloves garlic cr...",Mom's Stovetop Pork Ribs


In [31]:
recipies.to_csv('Dataset/Allrecipe.csv')
recipies.to_pickle("Dataset/Allrecipe.pkl")

# Playing with word2vec

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 
# sentences = open("101RecipesForTheDeepFryer.txt","r")
# gensim.utils.simple_preprocess(sentences)
# sentences.read()
# model = Word2Vec(sentences, size=100, window=50, min_count=1, workers=4)


2018-05-02 13:23:10,296 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin
2018-05-02 13:23:53,290 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin


In [141]:
# model.wv['eggs']
words = list(word_vectors.wv.vocab)
# pprint.pprint(words)
w1 = ['pork']
# word_vectors.wv['tomato']
word_vectors.wv.most_similar(positive=w1,topn=6)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('meat', 0.718697726726532),
 ('Pork', 0.6871966123580933),
 ('beef', 0.6601365208625793),
 ('chicken', 0.6541997790336609),
 ('meats', 0.6151372790336609),
 ('lamb', 0.6107682585716248)]